In [1]:
import sys
sys.path.append("..")
from data_prep.prep import MeterDataSet
import pandas as pd
import numpy as np
from webapp.utils.azure_utils import KeyVault, DataLake

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)
file_system = "energyhub"

In [3]:
meter = "electricity"
metadata_cols = ['building_id', 'site_id','sq_meter']
weather_cols = ['site_id', 'timestamp', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precipitation_depth_1_hr',
    'precipitation_depth_6_hr', 'sea_level_pressure', 'wind_direction',
    'wind_speed']

In [4]:
bad_building_dir = "/bad_buildings"
data = MeterDataSet(meter, metadata_cols, weather_cols)
bad_buildings = storage.pandas_read(
            file_system, directory=bad_building_dir, file_name="bad_buildings.csv")

In [5]:
# Treat nan for weather:
w_cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precipitation_depth_1_hr',
    'precipitation_depth_6_hr', 'sea_level_pressure', 'wind_direction',
    'wind_speed']
data.weather = data.fill_weather_na(w_cols = w_cols, fill_method = "linear")

In [6]:
# Treat nan for meter:
meter_cols = [meter]
data.meter = data.fill_meter_na(meter_col = meter_cols, fill_method = "linear")

In [7]:
data.meter.compute()

,timestamp,building_id,electricity
0,2016-02-01 03:00:00,Panther_parking_Lorriane,3.7207
1,2016-02-06 13:00:00,Panther_parking_Lorriane,2.1004
2,2016-02-14 05:00:00,Panther_parking_Lorriane,3.3006
3,2016-02-20 01:00:00,Panther_parking_Lorriane,3.4807
4,2016-02-29 11:00:00,Panther_parking_Lorriane,2.1604
...,...,...,...
7247,2017-01-26 07:00:00,Mouse_science_Micheal,1000.0000
7248,2017-01-26 10:00:00,Mouse_science_Micheal,1000.0000
7249,2017-01-26 13:00:00,Mouse_science_Micheal,1000.0000
7250,2017-01-26 16:00:00,Mouse_science_Micheal,1000.0000


In [14]:
data.weather = data.weather.compute()

In [15]:
data.meter = data.meter.compute()

In [16]:
data.metadata = data.metadata.compute()

In [17]:
# Merge weather, metadata and electricity:
data.merge()